In [3]:
import os

In [4]:
%pwd

'e:\\Mywork\\End To End Ml Projects\\End-To-End- Chiken Disease Deep-Learning-Project-Using-MLOPS-DVC-Pipeline-With-Deployments\\research'

In [5]:
os.chdir('../')

In [6]:
%pwd

'e:\\Mywork\\End To End Ml Projects\\End-To-End- Chiken Disease Deep-Learning-Project-Using-MLOPS-DVC-Pipeline-With-Deployments'

# Defing Return Type for Data Ingestion Which will be  defined in Config.entity.py file iside Entity folder 

In [7]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class DataIngestionReturnType:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

# Creating Configuration Manager Class in Congiguration.py File under src folder  

 This Class responsible for setting up all the configuration making artifacts folder 

In [8]:
from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml, create_directories

In [9]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])


    # return all the data ingestion related configuration 
    def get_data_ingestion_config(self) -> DataIngestionReturnType:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionReturnType(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir 
        )

        return data_ingestion_config
      

In [10]:
import os
import urllib.request as request
import zipfile
from cnnClassifier.logger import logger
from cnnClassifier.utils.common import get_size

# Creating Data Ingestion Class in Data Ingestion.py in Components Folder 

 This class is mainly responsible for retrieve data from github URL which is in Zip format 

In [11]:
class DataIngestion:
    def __init__(self, config: DataIngestionReturnType):
        self.config = config


    
    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url = self.config.source_URL,
                filename = self.config.local_data_file
            )
            logger.info(f"{filename} download! with following info: \n{headers}")
        else:
            logger.info(f"File already exists of size: {get_size(Path(self.config.local_data_file))}")  


    
    def extract_zip_file(self):
        """
        zip_file_path: str
        Extracts the zip file into the data directory
        Function returns None
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)

# Extracting Data in Training Pipeline 

In [13]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2024-08-31 20:04:22,392: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-08-31 20:04:22,395: INFO: common: yaml file: params.yaml loaded successfully]
[2024-08-31 20:04:22,406: INFO: common: created directory at: artifacts]
[2024-08-31 20:04:22,408: INFO: common: created directory at: artifacts/data_ingestion]
[2024-08-31 20:04:31,170: INFO: 1477750759: artifacts/data_ingestion/data.zip download! with following info: 
Connection: close
Content-Length: 11616915
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "adf745abc03891fe493c3be264ec012691fe3fa21d861f35a27edbe6d86a76b1"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: DB38:209313:4225D5:46545D:66D330F7
Accept-Ranges: bytes
Date: Sat, 31 Aug 2024 15:04:25 GMT
Via: 1.1 varnish
X-Served-By: cache-fjr990022-FJR
X-Cache: